In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelEncoder
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import re
import csv

In [2]:
try:
    # Read the CSV and skip bad lines
    raw_df = pd.read_csv("Suicide_Detection.csv", on_bad_lines='skip')
    print(raw_df.shape)
except Exception as e:
    print(f"An error occurred: {e}")

(232074, 3)


In [3]:
raw_df

,Unnamed: 0,text,class
0,2,Ex Wife Threatening SuicideRecently I left my ...,suicide
1,3,Am I weird I don't get affected by compliments...,non-suicide
2,4,Finally 2020 is almost over... So I can never ...,non-suicide
3,8,i need helpjust help me im crying so hard,suicide
4,9,"I’m so lostHello, my name is Adam (16) and I’v...",suicide
...,...,...,...
232069,348103,If you don't like rock then your not going to ...,non-suicide
232070,348106,You how you can tell i have so many friends an...,non-suicide
232071,348107,pee probably tastes like salty tea😏💦‼️ can som...,non-suicide
232072,348108,The usual stuff you find hereI'm not posting t...,suicide


In [4]:
raw_df.dtypes

Unnamed: 0     int64
text          object
class         object
dtype: object

In [5]:
df = raw_df.drop("Unnamed: 0", axis=1)
labelencoder = LabelEncoder()
df['class'] = labelencoder.fit_transform(df['class'])

In [6]:
df

,text,class
0,Ex Wife Threatening SuicideRecently I left my ...,1
1,Am I weird I don't get affected by compliments...,0
2,Finally 2020 is almost over... So I can never ...,0
3,i need helpjust help me im crying so hard,1
4,"I’m so lostHello, my name is Adam (16) and I’v...",1
...,...,...
232069,If you don't like rock then your not going to ...,0
232070,You how you can tell i have so many friends an...,0
232071,pee probably tastes like salty tea😏💦‼️ can som...,0
232072,The usual stuff you find hereI'm not posting t...,1


In [8]:
df.isnull().sum()

text     0
class    0
dtype: int64

In [13]:
# Download NLTK resources
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\sunio\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\sunio\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [9]:
# Preprocessing function
def preprocess_text(text):
    # Lowercasing
    text = text.lower()
    # Removing non-alphanumeric characters
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)
    # Tokenization
    tokens = text.split()
    # Stopwords removal
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if not word in stop_words]
    # Lemmatization
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(word) for word in tokens]
    return ' '.join(tokens)


In [17]:
# Preprocess the text data
df['text'] = df['text'].apply(preprocess_text)

# Feature Extraction
vectorizer = TfidfVectorizer(max_features=500)
X = vectorizer.fit_transform(df['text'])

print(X.shape)
print(X)
print(type(X))
print(vectorizer.get_feature_names())

# Assuming you have a labeled dataset (0 for non-suicide, 1 for suicide)
y = df['class']

# Splitting dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Model Training
model = LogisticRegression()
model.fit(X_train, y_train)

# Predictions
predictions = model.predict(X_test)

# Evaluation
print(classification_report(y_test, predictions))

(232074, 500)
  (0, 191)	0.1723289544843608
  (0, 106)	0.13133728601677205
  (0, 211)	0.1442020091426488
  (0, 272)	0.171000231010518
  (0, 442)	0.14192076589922167
  (0, 223)	0.06583967471918455
  (0, 147)	0.12691107301814453
  (0, 79)	0.17436482188429597
  (0, 168)	0.13103699261378524
  (0, 95)	0.17008028290421243
  (0, 63)	0.1891512618018274
  (0, 264)	0.12456825771272513
  (0, 402)	0.11933916702107412
  (0, 188)	0.18154877268622754
  (0, 96)	0.15641094230513394
  (0, 189)	0.19159704142696796
  (0, 414)	0.18407770417867103
  (0, 345)	0.100154255207746
  (0, 193)	0.19176740173862172
  (0, 469)	0.11350966379722487
  (0, 167)	0.08786732904054027
  (0, 321)	0.09736234213491547
  (0, 263)	0.13601380804826593
  (0, 240)	0.08488694853739831
  (0, 75)	0.13646427390684854
  :	:
  (232072, 204)	0.1158741384608553
  (232072, 296)	0.05730236947848372
  (232072, 109)	0.03947877671621315
  (232072, 223)	0.1386100524956253
  (232072, 168)	0.068966943193954
  (232072, 167)	0.13873809913520313
  (23

AttributeError: 'TfidfVectorizer' object has no attribute 'get_feature_names'

In [21]:
import pickle
# Save the model to disk using pickle
with open('suicide_model.pkl', 'wb') as file:
    pickle.dump(model, file)

# Save the vectorizer as well
with open('suicide_vectorizer.pkl', 'wb') as file:
    pickle.dump(vectorizer, file)


In [22]:
# After training the model...
feature_names = vectorizer.get_feature_names_out()
# Get the coefficients of the model
coefficients = model.coef_[0]
# Sort them in descending order and get the indices
sorted_indices = coefficients.argsort()[::-1]

print("Top indicators of depression based on model coefficients:")
for i in range(10):  # Let's look at the top 10 indicators
    # Feature names are the words, and coefficients are their corresponding weights
    print(f"{feature_names[sorted_indices[i]]}: {coefficients[sorted_indices[i]]}")


Top indicators of depression based on model coefficients:
suicide: 12.58061503149819
suicidal: 11.51691078453422
kill: 9.039636566026246
killing: 6.963092819642461
pill: 6.025798490385522
die: 5.872276304102521
ending: 5.361554332754518
end: 5.186448824155387
alive: 5.06513072328604
life: 4.9359017858258865


In [15]:
model.predict(vectorizer.transform(["I am very happy"]))

array([0])